# tuesday 2023-07-11 action items:
* ~~Debug Beirut, see where the source of decreased knn weight~~
* ~~re run an accessibility simulation, as well as a betweenness flow simulation for beirut~~
* ~~Once simulation accessibility results done, send to danie~~ addd flows to folder and send another email
* ~~create visuak maos for scenarios showing increase in reach.. Add to google slides~~
* ~~Homes to > transit - parks - amenities - schools~~
* ~~geometric - no turns, no elastic weight, no decay ....~~
* ~~la_extended16_manualFixed~~
* ~~Run Somerville~~
* ~~RUn LA~~
* ~~look at Andres' email about brooklyn , run a simulation using the old code in the workstation~~
* ~~look at Lui's email, see if I need to do anything about it~~
* ~~create OD pairing table for LA, send to Andres~~
* ~~RUN A FKOW MODEL FOR HOMES TO MOSKS, USING THE SAME PARAMETERS, WITH KNN , FOR ALL SCENARIOS, SEND TO DANIEL~~
* ~~CLip lui's dataset into the area that daniel chose, share with andres~~
*~~ FOR the clip, use this file "C:\Users\abdul\Dropbox (MIT)\115_NYCWalks\03_Data\02_Edited\NYC_Sidewalk_Edit_July\BK_clipping.geojson"~~
* ~~make sure the new node insertion code works and matches well~~

## effecient node edge creation
* ~~Make sure the functionality to construct nodes-edges works for a safey buffer~~
* ~~Isolate the node insertion code into script~~
* ~~commit script to my branch~~
* ~~email orion about it, mention how easy to integrate.~~
* ~~look into issue with redundant edges~~

## Testing
* ~~structure tests so its a one csv: first n rows are settings, next m rows are edges flows, with first column being same segment ids from network file~~
* ~~standard test case is four files: network.jeojson - origins.geojson - destination.jeojson - testflows.csv~~
* ~~function reads a csv into list of dicts: test settings and series of output.~~
* ~~build Harvard Square testflows.csv~~
~~* make sure this config runs on Harvard Square~~



# TODO
## Functionality List 1
* add visualization functionality to new version of Madina so testing is easier
* in network, add  connector segments as a separate layer (Maybe not very needed if origins and destinations visually are snapoped...)
* Commit visualization updates to github


## Testing
* digitize three test cases for manhattan, visualize OD snapping, show where differences (if any) come from between Rhino  and Python in the NYC case.
* Put maps into slides
* CHeck somerville, to see if we delete the same number of edges, with and without the effecient code "C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\Cities\Somerville\Data\Somerville_network_1000m_buffer.geojson"

## Functionality List 2
* build the betweenness simulation workflow, combine with accissibility
* include Logger as a class, and zonal would have an instance of that class. Logger handles event documentation, and captures output.
* for each origin, have an origin record, showing its knn weight, reach, gravity towards each of its destination, header rows showing settings and parameters
* for the network, columns of od flows, headed  by settings and parameters rows that explicitly detail units, weight type, calibration status, .. any relevant data, settings and parameters..
* set up a slideshow/diagram to show the relationship between different compoonents of the library.
* Set up a notebook to go over estimating flows from one origin to one destination, Document all relevant settings
* show how this generalizes to the pairings.csv, iterating over moodel settings, iterating over scenarios
* Make a finalized github commit including testing script, betweenness flow notebook. 
* Deploy to workstation.


## Functionality List 3
* given the simulation results, and the censor locations, find a way to calibrate counts and generate beta coeffecients (Beirut, SOmerville, NYC)
* enable Jupyter notebook server access from workstation.
* Figure out a way to do the "Cities" dropbox folder, people can add data through dropbox, run simulations through Jupyter Server, get results through Dropbox




## Running Cases
* Send output of LA to Nico when done
* send Brooklyn output when done
* be prepared to run entire NYC simulation, be mindful that daniel already sent more network parts


## Map Cleaning
https://www.dropbox.com/scl/fo/d6ugcyt3aanfb3yk4n9va/h?dl=0&rlkey=1c2giilvzoyygcxrlry7fscr9

In [ ]:
! pip install tqdm

In [ ]:
#from tqdm import tqdm
import sys
print(sys.version)

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import sys
import math


sys.path.append('../')
from madina.zonal.zonal import Zonal
from madina.una.betweenness import parallel_betweenness
from madina.una.elastic import get_elastic_weight

In [ ]:
for test_case in os.listdir("Test Cases"):
    # TODO: Check OS compatibility, ensure this is compatible with Unix systems..
    test_case_folder = "Test Cases" + "\\" + test_case + "\\"
    test_config = pd.read_csv(test_case_folder + "test_configs.csv")
    test_flows =  pd.read_csv(test_case_folder + "test_flows.csv")

    harvard_square = Zonal(projected_crs='EPSG:3857')

    harvard_square.load_layer(
        layer_name='streets',
        file_path=  test_case_folder + test_config.at[0, 'Network_File']
        )

    harvard_square.load_layer(
        layer_name=test_config.at[0, 'Origin_Name'],
        file_path= test_case_folder + test_config.at[0, 'Origin_File']
        )

    harvard_square.load_layer(
        layer_name=test_config.at[0, 'Destination_Name'],
        file_path= test_case_folder + test_config.at[0, 'Destination_File']
        )
    
    harvard_square.create_street_network(
        source_layer='streets', 
        discard_redundant_edges=True,
        node_snapping_tolerance=1.0
    )

    harvard_square.insert_node(
        layer_name=test_config.at[0, 'Origin_Name'], 
        label='origin', 
        weight_attribute=test_config.at[3, 'Origin_Weight']
    )

    harvard_square.insert_node(
        layer_name=test_config.at[0, 'Destination_Name'], 
        label='destination', 
        weight_attribute=test_config.at[3, 'Destination_Weight']
    )

    harvard_square.create_graph(light_graph=True, d_graph=True)

    node_gdf = harvard_square.network.nodes
    origin_gdf = node_gdf[node_gdf['type'] == 'origin']

    harvard_square.network.nodes["original_weight"] = harvard_square.network.nodes["weight"]


    # ["original_weight", "elastic_weight", "knn_weight"]

    for test_idx in test_config.index:
        harvard_square.network.turn_penalty_amount = test_config.at[test_idx, 'Turn penalty']
        harvard_square.network.turn_threshold_degree = test_config.at[test_idx, 'Turn threshold']

        if test_config.at[test_idx, 'Elastic_weights']:
            harvard_square.network.nodes["weight"] = harvard_square.network.nodes["original_weight"]
            get_elastic_weight(
                harvard_square.network,
                search_radius=test_config.at[test_idx, 'Radius'],
                detour_ratio=test_config.at[test_idx, 'Detour'],
                beta=test_config.at[test_idx, ' Beta '],
                decay=True, #test_config.at[test_idx, 'Decay'],
                #turn_penalty=test_config.at[test_idx, 'Turns'],
                turn_penalty=False,
            )
            for o_idx in origin_gdf.index:
                harvard_square.network.nodes.at[o_idx, 'weight'] =  harvard_square.network.nodes.at[o_idx, 'elastic_weight']


        return_dict = parallel_betweenness(
            harvard_square.network,
            search_radius=test_config.at[test_idx, 'Radius'],
            detour_ratio=test_config.at[test_idx, 'Detour'],
            decay=test_config.at[test_idx, 'Decay'], #if test['Elastic weights'] else True,
            decay_method=test_config.at[test_idx, 'Decay_Mode'],  # "power", "exponent"
            beta=test_config.at[test_idx, ' Beta '],
            path_detour_penalty="equal",  # "power", "exponent", "equal"
            origin_weights=False if type(test_config.at[test_idx, 'Origin_Weight']) != str else True,
            closest_destination=test_config.at[test_idx, 'Closest_destination'],
            destination_weights=False if type(test_config.at[test_idx, 'Destination_Weight']) != str  else True,    #or (test['Elastic weights'])
            # perceived_distance=False,
            num_cores=2,
            light_graph=True,
            turn_penalty=test_config.at[test_idx, 'Turns'],
        )
        simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()
        test_flow = test_flows[test_config.at[test_idx, 'test_name']].sum()

        print (test_config.loc[test_idx])
        print (f"{test_config.at[test_idx, 'test_name']}\t\t{simulated_sum_of_flow = }\t test flow = { test_flow }\t difference = {simulated_sum_of_flow - test_flow}\t similarity {1-(simulated_sum_of_flow - test_flow)/ test_flow:.2%}")
    print ("DOne Case...")
    break

# COnstructing the Manhattan Case..
## Don't forget to check for node insertion, set  return_all=False in the spatial index

In [ ]:
import time
start = time.time()

buildings_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\Home_PT_6538.geojson"
subway_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\Metro_PT_6538.geojson"
network_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\network_clipped_dupremovedAS.geojson"

import sys
sys.path.append('../')
from madina.zonal.zonal import Zonal


harvard_square = Zonal()

print(f"{(time.time()-start)*1000:6.2f}ms\t imports done, object created")
start = time.time()

#harvard_square.load_layer(
#    layer_name='streets',
#    file_path=network_file
#    )


print(f"{(time.time()-start)*1000:6.2f}ms\t street data loaded")
start = time.time()

harvard_square.load_layer(
    layer_name='buildings',
    file_path=buildings_file
    )

buildings_gdf = harvard_square.layers['buildings'].gdf
harvard_square.layers['buildings'].gdf = buildings_gdf[~buildings_gdf['FID'].isin([27967, 9140, 3974])]

print(f"{(time.time()-start)*1000:6.2f}ms\t building data loaded")
start = time.time()

harvard_square.load_layer(
    layer_name='subway',
    file_path=subway_file
    )

print(f"{(time.time()-start)*1000:6.2f}ms\t subway data loaded")

In [ ]:
start = time.time()

harvard_square.create_street_network(
    source_layer='streets', 
    discard_redundant_edges=False, 
    node_snapping_tolerance=0
)

print(f"{(time.time()-start)*1000:6.2f}ms\t street network created")
start = time.time()

harvard_square.insert_node(
    layer_name='buildings', 
    label='origin', 
    weight_attribute='TotalPop'
)


print(f"{(time.time()-start)*1000:6.2f}ms\t origins insertes")
start = time.time()

harvard_square.insert_node(
    layer_name='subway', 
    label='destination', 
    weight_attribute='line_ent_st'
)

print(f"{(time.time()-start)*1000:6.2f}ms\t destinations insertes")


start = time.time()

harvard_square.create_graph(light_graph=True, d_graph=True)

print(f"{(time.time()-start)*1000:6.2f}ms\t graph created insertes")
start = time.time()

return_dict = parallel_betweenness(
    harvard_square.network,
    search_radius=800,
    detour_ratio=1.15,
    decay=False,
    decay_method='exponent',  # "power", "exponent"
    beta=0.004,
    path_detour_penalty="equal",  # "power", "exponent", "equal"
    origin_weights=True,
    closest_destination=False,
    destination_weights=True, 
    # perceived_distance=False,
    num_cores=8,
    light_graph=True,
    turn_penalty=False,
)
simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()

print(f"{(time.time()-start)*1000:6.2f}ms\t Betweenness estimated")
start = time.time()

In [ ]:
joined_results = harvard_square.layers['streets'].gdf.join(harvard_square.network.edges[['parent_street_id', 'betweenness']].set_index('parent_street_id'))
joined_results[['__GUID', 'betweenness', 'geometry']].to_csv('2023-07-07 manhattan betweenness flow test.csv')
joined_results[['__GUID', 'betweenness', 'geometry']].to_file('2023-07-07 manhattan betweenness flow test.geoJSON', driver="GeoJSON")

In [ ]:
harvard_square.create_map(
    [
        #{
            #'layer': 'streets',
            #'color': [125, 125, 125],
        #},
        {
            'gdf': harvard_square.layers['subway'].gdf,
            'color_by_attribute': 'line_ent_st',
            "color_method": 'gradient'
        }
    ],
    save_as="map.html"
)

## Need to instal pydeck jupyter nb extension

`jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck`

`jupyter nbextension enable --sys-prefix --py pydeck`
